# **Custom Legal Bert**

## Install

Install required packages and dependencies:

In [ ]:
# !pip install -r requirements.txt

Install transformers from source (required for tokenizers dependencies):

In [ ]:
# !pip install git+https://github.com/huggingface/transformers

In [ ]:
# !pip install scikit-learn

## Creating Training Data

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [3]:
import os

df = pd.DataFrame()
for file in os.listdir("./data/ToS/Sentences"):
    if file.endswith(".txt"):
        print(file)
        sentence_path = './data/ToS/Sentences/' + file
        label_path = './data/ToS/Labels/' + file
        if df.shape[0] == 0:
            df = pd.read_csv(sentence_path, sep='\n', names=['sentences'])
            df['labels'] = pd.read_csv(label_path, sep='\n', names=['labels'])
            df['labels_converted'] = np.where(df['labels'] == -1, 0, 1) ## binary classification
        else:
            df_new = pd.read_csv(sentence_path, sep='\n', names=['sentences'])
            df_new['labels'] = pd.read_csv(label_path, sep='\n', names=['labels'])
            df_new['labels_converted'] = np.where(df_new['labels'] == -1, 0, 1)
            df = pd.concat([df, df_new], ignore_index=False, axis=0)
        

Viber.txt
Nintendo.txt
Tinder.txt
Dropbox.txt
Microsoft.txt
Betterpoints_UK.txt
Airbnb.txt
musically.txt
Crowdtangle.txt
TripAdvisor.txt
Deliveroo.txt
Moves-app.txt
Spotify.txt
Supercell.txt
9gag.txt
Booking.txt
Headspace.txt
Fitbit.txt
Syncme.txt
Vimeo.txt
Oculus.txt
Endomondo.txt
Instagram.txt
LindenLab.txt
WorldOfWarcraft.txt
YouTube.txt
Academia.txt
Yahoo.txt
WhatsApp.txt
Google.txt
Zynga.txt
Facebook.txt
Amazon.txt
Vivino.txt
Netflix.txt
PokemonGo.txt
Skype.txt
Snap.txt
eBay.txt
Masquerade.txt
Twitter.txt
LinkedIn.txt
Skyscanner.txt
Duolingo.txt
TrueCaller.txt
Uber.txt
Rovio.txt
Atlas.txt
Evernote.txt
Onavo.txt


In [4]:
df.shape

(9414, 3)

In [5]:
df

,sentences,labels,labels_converted
0,thanks for sending us good vibes by using the ...,-1,0
1,"you may be surprised , but we will refer to al...",-1,0
2,"the terms of use -lrb- or , the `` terms '' -r...",-1,0
3,the language of the terms will seem legal -lrb...,-1,0
4,"when you use our services , in addition to enj...",1,1
...,...,...,...
142,the failure of onavo to enforce any right or p...,-1,0
143,the section headings in the agreement are incl...,-1,0
144,"`` including '' , whether capitalized or not ,...",-1,0
145,this agreement may not be assigned by you with...,-1,0


In [6]:
df.isna().sum()

sentences           0
labels              0
labels_converted    0
dtype: int64

In [7]:
clauses_df = df.drop(columns=['labels'])

In [8]:
clauses_df.head()

,sentences,labels_converted
0,thanks for sending us good vibes by using the ...,0
1,"you may be surprised , but we will refer to al...",0
2,"the terms of use -lrb- or , the `` terms '' -r...",0
3,the language of the terms will seem legal -lrb...,0
4,"when you use our services , in addition to enj...",1


In [9]:
clauses_df.shape

(9414, 2)

In [10]:
clauses_df.rename(columns={'labels_converted':'label'}, inplace=True)

In [11]:
clauses_df.head()

,sentences,label
0,thanks for sending us good vibes by using the ...,0
1,"you may be surprised , but we will refer to al...",0
2,"the terms of use -lrb- or , the `` terms '' -r...",0
3,the language of the terms will seem legal -lrb...,0
4,"when you use our services , in addition to enj...",1


In [12]:
clauses_df.to_csv('./data/ToS/tos_clauses.csv', index=False)

## Terms of Service

#### Compute pretrain loss
To compute per example/average pretrain loss across the full dataset, run the `run_glue.py` script with the arguments specified in the example.
- Pass a file containing the full dataset to `validation_file`.
- Pass `ptl=True`. 
- The script requires a `train_file`, but does not use it when `ptl=True`, so the particular file passed is not important in this case.

Running the `run_glue.py` script with `ptl=True` writes per example pretrain loss (order matches order of examples in `validation_file`) to the file `per_ex_pretrain_loss.csv` in `output_dir`. The script also prints the average pretrain loss across `validation_file` examples.


*Calculate domain specificity (DS) scores*

To calculate the domain specificity (DS) score of a task, take the difference in average pretrain loss on BERT (double) and Legal-BERT $$\overline{L}_{BERT (double)} - \overline{L}_{Custom Legal-BERT}$$

It is also possible to use the script to calculate the DS score of a specific task example $i$ by taking the difference in the example $i$ pretrain loss on BERT (double) and Legal-BERT $$L^{(i)}_{BERT (double)} - L^{(i)}_{Custom Legal-BERT}$$

Set environment variable to disable tokenizers parallelism:

In [1]:
%env TOKENIZERS_PARALLELISM=false

env: TOKENIZERS_PARALLELISM=false


In [13]:
# Download model from Hugging Face model repository
!python classification/run_glue.py \
  --model_name_or_path zlucia/custom-legalbert \
  --train_file data/ToS/tos_clauses.csv \
  --validation_file data/ToS/tos_clauses.csv \
  --ptl=True \
  --max_seq_length 128 \
  --output_dir logs/ToS/custom-legalbert \
  --overwrite_output_dir

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
Moving 0 files to the new cache system
0it [00:00, ?it/s]
10/14/2022 20:33:06 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0distributed training: False, 16-bits training: False
10/14/2022 20:33:06 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluatio

[INFO|modeling_utils.py:2156] 2022-10-14 20:33:08,206 >> loading weights file pytorch_model.bin from cache at /Users/lavina/.cache/huggingface/hub/models--zlucia--custom-legalbert/snapshots/fd49a135d7b327a315e3ffea31c2be1b40685315/pytorch_model.bin
[WARNING|modeling_utils.py:2597] 2022-10-14 20:33:09,240 >> Some weights of the model checkpoint at zlucia/custom-legalbert were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining m

In [ ]:
# !pip install importlib-metadata==4.13. 0 ##‘AttributeError: EntryPoints’ object has no attribute ‘get’

#### Finetune

To finetune on the dataset, run the `run_glue.py` script with the arguments specified in the example. The hyperparameters specified are the same as those from the paper.
- Pass a file containing the train split to `train_file` and a file containing the split to evaluate/predict on (dev or test split) to `validation_file`.
- Pass `do_train` to train on `train_file`, `do_eval` to evaluate on `validation_file`, and `do_predict` to predict on `validation_file`. 

Running the `run_glue.py` script with `do_train` and `do_eval` trains the specified model on `train_file`, evaluates the trained model on `validation_file`, and writes the trained model/tokenizer files and the evaluation results to the file `eval_results.txt` in `output_dir`. Passing `do_predict` writes the class label predictions on `validation_file` to the file `predictions.csv` in `output_dir`. The script also prints the evaluation results on `validation_file` (evaluation F1, evaluation loss etc.).

### Split training data into two sets

In [14]:
train, dev = train_test_split(clauses_df, test_size=0.2, random_state=42, stratify=clauses_df[['label']])

In [15]:
train.head()

,sentences,label
5,content license and intellectual property rights,0
232,reactivated skype credit is not refundable .,0
196,spotify may change the price for the paid subs...,1
18,the term of your licenses under this eula shal...,0
181,the arbitrator may award declaratory or injunc...,0


In [17]:
dev.head()

,sentences,label
50,uber reserves the right to withhold or deduct ...,0
208,niantic 's failure to enforce any right or pro...,0
275,14.3 if you feel that any member you interact ...,0
182,blizzard entertainment has the right to obtain...,0
151,myfitnesspal does not -lrb- i -rrb- guarantee ...,0


In [18]:
train.to_csv("./data/ToS/tos_clauses_train.csv", index=False)
dev.to_csv("./data/ToS/tos_clauses_dev.csv", index=False)

In [19]:
# Download model from Hugging Face model repository
!python classification/run_glue.py \
  --model_name_or_path zlucia/custom-legalbert \
  --train_file data/ToS/tos_clauses_train.csv \
  --validation_file data/ToS/tos_clauses_dev.csv \
  --do_train \
  --do_eval \
  --evaluation_strategy steps \
  --max_seq_length 128 \
  --per_device_train_batch_size=16 \
  --learning_rate=1e-5 \
  --num_train_epochs=2.0 \
  --output_dir logs/ToS/custom-legalbert \
  --overwrite_output_dir \
  --logging_steps 50

10/14/2022 22:34:45 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0distributed training: False, 16-bits training: False
10/14/2022 22:34:45 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=50,
evaluation_strategy=steps,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,

[INFO|modeling_utils.py:2156] 2022-10-14 22:34:47,139 >> loading weights file pytorch_model.bin from cache at /Users/lavina/.cache/huggingface/hub/models--zlucia--custom-legalbert/snapshots/fd49a135d7b327a315e3ffea31c2be1b40685315/pytorch_model.bin
[WARNING|modeling_utils.py:2597] 2022-10-14 22:34:48,413 >> Some weights of the model checkpoint at zlucia/custom-legalbert were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining m


 41%|█████████████████▎                        | 97/236 [07:07<10:13,  4.41s/it]


 82%|█████████████████████████████████▋       | 194/236 [14:13<03:05,  4.43s/it]


100%|█████████████████████████████████████████| 236/236 [17:18<00:00,  3.63s/it]

Downloading: 4.38kB [00:00, 805kB/s]                                            
10/14/2022 23:14:44 - INFO - /Users/lavina/opt/anaconda3/envs/CustomLegalBERT/lib/python3.7/site-packages/datasets/metric.py -   Removing /Users/lavina/.cache/huggingface/metrics/f1/default/default_experiment-1-0.arrow
                                                                                
{'eval_loss': 0.28940603137016296, 'eval_f1': 0.0, 'eval_runtime': 1043.7882, 'eval_samples_per_second': 1.804, 'eval_steps_per_second': 0.226, 'epoch': 0.11}
100%|█████████████████████████████████████████| 236/236 [17:19<00:00,  3.63s/it]
{'loss': 0.2817, 'learning_rate': 8.938428874734608e-06, 'epoch': 0.21}         
 11%|███▉                                 | 100/942 [1:02:55<6:11:57, 26.51s/it][INFO|trainer.py:726] 2022-10-14 23:37:44,803 >> The following columns in the evaluation set don't have a corresponding argument in `Ber

...

 58%|███████████████████████▋                 | 136/236 [09:52<07:18,  4.39s/it]


 99%|████████████████████████████████████████▍| 233/236 [16:58<00:13,  4.39s/it]


100%|█████████████████████████████████████████| 236/236 [17:09<00:00,  3.57s/it]10/15/2022 21:54:29 - INFO - /Users/lavina/opt/anaconda3/envs/CustomLegalBERT/lib/python3.7/site-packages/datasets/metric.py -   Removing /Users/lavina/.cache/huggingface/metrics/f1/default/default_experiment-1-0.arrow
                                                                                
{'eval_loss': 0.1568325012922287, 'eval_f1': 0.7851851851851852, 'eval_runtime': 1035.1003, 'eval_samples_per_second': 1.819, 'eval_steps_per_second': 0.228, 'epoch': 1.8}
100%|█████████████████████████████████████████| 236/236 [17:10<00:00,  3.57s/it]
{'loss': 0.1038, 'learning_rate': 4.45859872611465e-07, 'epoch': 1.91}          
 96%|████████████████████████████████████▎ | 900/942 [23:41:58<18:44, 26.78s/it][INFO|trainer.py:726] 2022-10-15 22:16:48,599 >> The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentences. 

 74%|██████████████████████████████▍          | 175/236 [12:45<04:27,  4.39s/it]


100%|█████████████████████████████████████████| 236/236 [17:10<00:00,  3.57s/it]10/15/2022 22:34:05 - INFO - /Users/lavina/opt/anaconda3/envs/CustomLegalBERT/lib/python3.7/site-packages/datasets/metric.py -   Removing /Users/lavina/.cache/huggingface/metrics/f1/default/default_experiment-1-0.arrow
                                                                                
{'eval_loss': 0.15546682476997375, 'eval_f1': 0.7851851851851852, 'eval_runtime': 1036.5083, 'eval_samples_per_second': 1.817, 'eval_steps_per_second': 0.228, 'epoch': 1.91}
100%|██████████████████████████████████████| 942/942 [24:17:54<00:00, 24.50s/it][INFO|trainer.py:1852] 2022-10-15 22:52:43,817 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)


{'train_runtime': 87474.1387, 'train_samples_per_second': 0.172, 'train_steps_per_second': 0.011, 'train_loss': 0.16293498513045584, 'epoch': 2.0}
100%|██████████████████████████████████████| 942/942 [24:17:54<00:00, 92.86s/it]
[I